In [1]:
import json
import collections
import logging
import copy

In [2]:
logging.basicConfig(format='%(message)s', level=logging.DEBUG)
logging.debug('This message should appear on the console')
logging.info('So should this')
logging.warning('And this, too')

This message should appear on the console
So should this
And this, too


In [3]:
bounding_school_parsed_path = r'bounding_school_parsed.json'

In [4]:
with open(bounding_school_parsed_path, 'r') as read_file:
    bounding_school = json.load(read_file)

In [5]:
# bounding_local_parsed_path = 'bounding_local_parsed.json'
# bounding_university_parsed_path = 'bounding_university_parsed.json'
# bounding_college_parsed_path = 'bounding_college_parsed.json'

In [6]:
# with open(bounding_local_parsed_path, 'r') as read_file:
#     bounding_local = json.load(read_file)
# with open(bounding_university_parsed_path, 'r') as read_file:
#     bounding_university = json.load(read_file)
# with open(bounding_college_parsed_path, 'r') as read_file:
#     bounding_college = json.load(read_file)

In [7]:
def adjusttotuples(alist):
    import copy
    alist_copy = copy.deepcopy(alist)
    for bound in alist_copy:
        if bound['type'] == 'relation':
            for waylist in bound['nodelist']:
                for pairindex, pair in enumerate(waylist):
                    waylist[pairindex] = tuple(pair)
        if bound['type'] == 'way':
            for pairindex, pair in enumerate(bound['nodelist']):
                bound['nodelist'][pairindex] = tuple(pair)
    return alist_copy

In [8]:
def arbitraryflatten(iterable):
    iterator, sentinel, stack = iter(iterable), object(), []
    while True:
        value = next(iterator, sentinel)
        if value is sentinel:
            if not stack:
                break
            iterator = stack.pop()
        elif isinstance(value, str):
            yield value
        else:
            try:
                new_iterator = iter(value)
            except TypeError:
                yield value
            else:
                stack.append(iterator)
                iterator = new_iterator

In [9]:
def checkshared(alist):
    import copy
    namesdict = {}
    passtest = True
    for bound in alist:
        nodelist = bound['nodelist']
        thename = bound['name']
        if thename not in namesdict:
            namesdict[thename] = copy.deepcopy(bound)
        else:
            #compare how far apart they are
            fromnamesdict = copy.deepcopy(namesdict[thename]['nodelist'])
            frombound = copy.deepcopy(bound['nodelist'])
            fromnamesdictflattened = list(arbitraryflatten(fromnamesdict))
            fromnamesdictflattenedlat = [element for i, element in enumerate(fromnamesdictflattened) if i%2==0]
            fromnamesdictflattenedlon = [element for i, element in enumerate(fromnamesdictflattened) if i%2!=0]
            fromnamesdictzipped = list(zip(fromnamesdictflattenedlat, fromnamesdictflattenedlon))
            
            fromboundflattened = list(arbitraryflatten(frombound))
            fromboundflattenedlat = [element for i, element in enumerate(fromboundflattened) if i%2==0]
            fromboundflattenedlon = [element for i, element in enumerate(fromboundflattened) if i%2!=0]
            fromboundzipped = list(zip(fromboundflattenedlat, fromboundflattenedlon))
            
            aset = set()
            for el in fromboundzipped:
                aset.add(el)
            shared = aset.intersection(fromnamesdictzipped)
            
            if shared:
                passtest = False

    return passtest

In [10]:
def combineshared(alist):
    import copy
    namesdict = {}
    finallist = []
    copiedalist = copy.deepcopy(alist)
    for bound in copiedalist:
        nodelist = bound['nodelist']
        thename = bound['name']
        if thename not in namesdict:
            namesdict[thename] = copy.deepcopy(bound)
            finallist.append(namesdict[thename])
        else:
            #compare how far apart they are
            fromnamesdict = copy.deepcopy(namesdict[thename]['nodelist'])
            frombound = copy.deepcopy(bound['nodelist'])
            fromnamesdictflattened = list(arbitraryflatten(fromnamesdict))
            fromnamesdictflattenedlat = [element for i, element in enumerate(fromnamesdictflattened) if i%2==0]
            fromnamesdictflattenedlon = [element for i, element in enumerate(fromnamesdictflattened) if i%2!=0]
            fromnamesdictzipped = list(zip(fromnamesdictflattenedlat, fromnamesdictflattenedlon))
            
            fromboundflattened = list(arbitraryflatten(frombound))
            fromboundflattenedlat = [element for i, element in enumerate(fromboundflattened) if i%2==0]
            fromboundflattenedlon = [element for i, element in enumerate(fromboundflattened) if i%2!=0]
            fromboundzipped = list(zip(fromboundflattenedlat, fromboundflattenedlon))
            
            aset = set()
            for el in fromboundzipped:
                aset.add(el)
            shared = aset.intersection(fromnamesdictzipped)

            if shared:
                indexoffirstbound = finallist.index(namesdict[thename])
                if finallist[indexoffirstbound]['type'] == 'way':
                    finallist[indexoffirstbound]['nodelist'] = [finallist[indexoffirstbound]['nodelist']]
                if bound['type'] == 'way':
                    bound['nodelist'] = [bound['nodelist']]
                finallist[indexoffirstbound]['nodelist'].extend(copy.deepcopy(bound['nodelist']))
                bound['type'] = 'relation'
                namesdict[thename]['type'] = 'relation'
                finallist[indexoffirstbound]['type'] = 'relation'
                        
            else:
                finallist.append(copy.deepcopy(bound))
                #pop the other one
    return finallist

In [11]:
#redo until passes
def surelycombinedshared(alist):
    redocount = 0
    while not checkshared(alist):
        alist = combineshared(alist)
        redocount += 1
    logging.debug(redocount)
    return alist

In [12]:
try_bounding_school = surelycombinedshared(bounding_school)

3


In [13]:
try_tuple_bounding_school = adjusttotuples(try_bounding_school)

In [14]:
# try_bounding_college = surelycombinedshared(bounding_college)
# try_bounding_university = surelycombinedshared(bounding_university)
# try_bounding_local = surelycombinedshared(bounding_local)

In [15]:
# try_tuple_bounding_college = adjusttotuples(try_bounding_college)
# try_tuple_bounding_university = adjusttotuples(try_bounding_university)
# try_tuple_bounding_local = adjusttotuples(try_bounding_local)

In [16]:
def forwardreversecomparison(thenodelist):
    import copy
    forwardcount = 0
    reversecount = 0
    
    newnodelistforward = copy.deepcopy(thenodelist)
    safewaylistsforward = []

    firstmembersforward = []
    lastmembersforward = []

    for index, waylist in enumerate(newnodelistforward):
        for wayindex, element in enumerate(waylist):
            waylist[wayindex] = tuple(element)

    for index, waylist in enumerate(newnodelistforward):
        if waylist[-1] == waylist[0]:
            safewaylistsforward.append(copy.deepcopy(waylist))
            newnodelistforward.pop(index)
        else:
            firstmembersforward.append(copy.deepcopy(waylist[0]))
            lastmembersforward.append(copy.deepcopy(waylist[-1]))

    nextfirstmembersforward = []
    nextlastmembersforward = []

    while nextfirstmembersforward != firstmembersforward and nextlastmembersforward != lastmembersforward:
        nextfirstmembersforward = []
        nextlastmembersforward = []
        for index, waylist in enumerate(newnodelistforward):         
            if waylist[-1] == waylist[0]:
                safewaylistsforward.append(copy.deepcopy(waylist))
                newnodelistforward.pop(index)
            else:
                nextfirstmembersforward.append(copy.deepcopy(waylist[0]))
                nextlastmembersforward.append(copy.deepcopy(waylist[-1]))

        firstmembersforward = copy.deepcopy(nextfirstmembersforward)
        lastmembersforward = copy.deepcopy(nextlastmembersforward)

        asetforward = set()
        for el in firstmembersforward:
            asetforward.add(el)
        sharedforward = asetforward.intersection(lastmembersforward)

        while sharedforward != set():
            forwardcount += 1
            shareforward = sharedforward.pop()
            index1forward = firstmembersforward.index(shareforward)
            index2forward = lastmembersforward.index(shareforward)

            newnodelistforward[index1forward].pop(0)

            newnodelistforward[index2forward].extend(copy.deepcopy(newnodelistforward[index1forward]))

            newnodelistforward.pop(index1forward)

            firstmembersforward = []
            lastmembersforward = []
            for index, waylist in enumerate(newnodelistforward):         
                if waylist[-1] == waylist[0]:
                    safewaylistsforward.append(copy.deepcopy(waylist))
                    newnodelistforward.pop(index)
                else:
                    firstmembersforward.append(copy.deepcopy(waylist[0]))
                    lastmembersforward.append(copy.deepcopy(waylist[-1]))
            asetforward = set()
            for el in firstmembersforward:
                asetforward.add(el)
            sharedforward = asetforward.intersection(lastmembersforward)
    newnodelistreverse = copy.deepcopy(thenodelist)
    safewaylistsreverse = []

    firstmembersreverse = []
    lastmembersreverse = []

    for index, waylist in enumerate(newnodelistreverse):
        for wayindex, element in enumerate(waylist):
            waylist[wayindex] = tuple(element)

    for index, waylist in enumerate(newnodelistreverse):
        if waylist[-1] == waylist[0]:
            safewaylistsreverse.append(copy.deepcopy(waylist))
            newnodelistreverse.pop(index)
        else:
            firstmembersreverse.append(copy.deepcopy(waylist[0]))
            lastmembersreverse.append(copy.deepcopy(waylist[-1]))

    nextfirstmembersreverse = []
    nextlastmembersreverse = []

    while nextfirstmembersreverse != firstmembersreverse and nextlastmembersreverse != lastmembersreverse:
        nextfirstmembersreverse = []
        nextlastmembersreverse = []
        for index, waylist in enumerate(newnodelistreverse):         
            if waylist[-1] == waylist[0]:
                safewaylistsreverse.append(copy.deepcopy(waylist))
                newnodelistreverse.pop(index)
            else:
                nextfirstmembersreverse.append(copy.deepcopy(waylist[0]))
                nextlastmembersreverse.append(copy.deepcopy(waylist[-1]))

        firstmembersreverse = copy.deepcopy(nextfirstmembersreverse)
        lastmembersreverse = copy.deepcopy(nextlastmembersreverse)

        asetreverse = set()
        for el in firstmembersreverse:
            asetreverse.add(el)
        sharedreverse = asetreverse.intersection(lastmembersreverse)

        while sharedreverse != set():
            reversecount += 1
            sharereverse = sharedreverse.pop()
            index1reverse = firstmembersreverse.index(sharereverse)
            index2reverse = lastmembersreverse.index(sharereverse)

            newnodelistreverse[index1reverse].pop(0)

            newnodelistreverse[index2reverse].extend(copy.deepcopy(newnodelistreverse[index1reverse]))

            newnodelistreverse[index2reverse] = newnodelistreverse[index2reverse][::-1]

            newnodelistreverse.pop(index1reverse)

            firstmembersreverse = []
            lastmembersreverse = []
            for index, waylist in enumerate(newnodelistreverse):         
                if waylist[-1] == waylist[0]:
                    safewaylistsreverse.append(copy.deepcopy(waylist))
                    newnodelistreverse.pop(index)
                else:
                    firstmembersreverse.append(copy.deepcopy(waylist[0]))
                    lastmembersreverse.append(copy.deepcopy(waylist[-1]))
            asetreverse = set()
            for el in firstmembersreverse:
                asetreverse.add(el)
            sharedreverse = asetreverse.intersection(lastmembersreverse)


    if forwardcount >= reversecount:
        newnodelist = newnodelistforward
        safewaylists = safewaylistsforward
        bestcount = forwardcount
    else:
        newnodelist = newnodelistreverse
        safewaylists = safewaylistsreverse
        bestcount = reversecount
    return newnodelist, safewaylists, bestcount

In [17]:
def mergeways(alist):
    import copy
    finalnodelist = []
    alist_copy = copy.deepcopy(alist)
    for bound in alist_copy:
        if bound['type'] == 'relation':
            newnodelist, safewaylists, bestcount = forwardreversecomparison(bound['nodelist'])
            #######################
            firstmembers = []
            lastmembers = []
            for index, waylist in enumerate(newnodelist):
                if waylist[-1] == waylist[0]:
                    safewaylists.append(copy.deepcopy(waylist))
                    newnodelist.pop(index)
                else:
                    firstmembers.append(copy.deepcopy(waylist[0]))
                    lastmembers.append(copy.deepcopy(waylist[-1]))
                    
                    
            nextfirstmembers = []
            nextlastmembers = []
            #########################
            while nextfirstmembers != firstmembers and nextlastmembers != lastmembers:
               
                nextfirstmembers = []
                nextlastmembers = []
                for index, waylist in enumerate(newnodelist):         
                    if waylist[-1] == waylist[0]:
                        safewaylists.append(copy.deepcopy(waylist))
                        newnodelist.pop(index)
                    else:
                        nextfirstmembers.append(copy.deepcopy(waylist[0]))
                        nextlastmembers.append(copy.deepcopy(waylist[-1]))
                
                firstmembers = copy.deepcopy(nextfirstmembers)
                lastmembers = copy.deepcopy(nextlastmembers)
                
                ######################## firstmembers
                
                firstmemdups = [item for item, count in collections.Counter(firstmembers).items() if count > 1]
                while firstmemdups:
                    first = firstmemdups.pop()
                    
                    index1 = firstmembers.index(first)
                    index2 = [i for i, el in enumerate(firstmembers) if el == first][1]
                    
                    #$$$$$$$$$$$$ try both
                    templist1 = copy.deepcopy(newnodelist)
                    templist1[index1] = templist1[index1][::-1]
                    
                    templist1, templist1safe, templist1bestcount = forwardreversecomparison(templist1)
                    
                    templist2 = copy.deepcopy(newnodelist)
                    templist2[index2] = templist2[index2][::-1]
                    
                    templist2, templist2safe, templist2bestcount = forwardreversecomparison(templist2)
                    
                    if templist1bestcount >= templist2bestcount:
                        newnodelist = templist1
                        safewaylists.extend(templist1safe)
                    else:
                        newnodelist = templist2
                        safewaylists.extend(templist2safe)
                    
                    firstmembers = []
                    lastmembers = []
                    for index, waylist in enumerate(newnodelist):         
                        if waylist[-1] == waylist[0]:
                            safewaylists.append(copy.deepcopy(waylist))
                            newnodelist.pop(index)
                        else:
                            firstmembers.append(copy.deepcopy(waylist[0]))
                            lastmembers.append(copy.deepcopy(waylist[-1]))
                    firstmemdups = [item for item, count in collections.Counter(firstmembers).items() if count > 1]
                 
                
                #########
                firstmembers = []
                lastmembers = []
                for index, waylist in enumerate(newnodelist):         
                    if waylist[-1] == waylist[0]:
                        safewaylists.append(copy.deepcopy(waylist))
                        newnodelist.pop(index)
                    else:
                        firstmembers.append(copy.deepcopy(waylist[0]))
                        lastmembers.append(copy.deepcopy(waylist[-1]))
                ######################## lastmembers
                lastmemedups = [item for item, count in collections.Counter(lastmembers).items() if count > 1]
                while lastmemedups:
                    
                    last = lastmemedups.pop()
                    
                    index1 = lastmembers.index(last)
                    index2 = [i for i, el in enumerate(lastmembers) if el == last][1]


                    #$$$$$$$$$$$$ try both
                    templist1 = copy.deepcopy(newnodelist)
                    templist1[index1] = templist1[index1][::-1]
                    
                    templist1, templist1safe, templist1bestcount = forwardreversecomparison(templist1)
                    
                    templist2 = copy.deepcopy(newnodelist)
                    templist2[index2] = templist2[index2][::-1]
                    
                    templist2, templist2safe, templist2bestcount = forwardreversecomparison(templist2)
                    
                    if templist1bestcount >= templist2bestcount:
                        newnodelist = templist1
                        safewaylists.extend(templist1safe)
                    else:
                        newnodelist = templist2
                        safewaylists.extend(templist2safe)

                    firstmembers = []
                    lastmembers = []
                    for index, waylist in enumerate(newnodelist):         
                        if waylist[-1] == waylist[0]:
                            safewaylists.append(copy.deepcopy(waylist))
                            newnodelist.pop(index)
                        else:
                            firstmembers.append(copy.deepcopy(waylist[0]))
                            lastmembers.append(copy.deepcopy(waylist[-1]))
                    lastmemedups = [item for item, count in collections.Counter(lastmembers).items() if count > 1]    
            newsafewaylists = []
            for safe in safewaylists:
              if safe not in newsafewaylists:
                newsafewaylists.append(safe)
            safewaylists = newsafewaylists
            newnodelist.extend(safewaylists)
            finalnodelist.append(newnodelist)
        if bound['type'] == 'way':
            nodelist = bound['nodelist']
            newnodelist = copy.deepcopy(nodelist)
            finalnodelist.append(newnodelist)
    return finalnodelist

In [18]:
def applyandcopymergeways(alist):
    import copy
    alist_copy = copy.deepcopy(alist)
    allnodelists = mergeways(alist_copy)
    logging.debug(len(allnodelists))
    for nodelistindex, nodelist in enumerate(allnodelists):
        alist_copy[nodelistindex]['nodelist'] = nodelist
        if len(nodelist) > 1:
            alist_copy[nodelistindex]['type'] = 'relation'
            try:
                alist_copy[nodelistindex]['nodelist'][0][0][0]
            except TypeError as notfullynested:
                alist_copy[nodelistindex]['type'] = 'way'
        elif len(nodelist) == 1 and alist_copy[nodelistindex]['type'] == 'relation':
            alist_copy[nodelistindex]['nodelist'] = nodelist[0]
            alist_copy[nodelistindex]['type'] = 'way'
    return alist_copy

In [27]:
new_bounding_school = applyandcopymergeways(try_tuple_bounding_school)

520772


In [28]:
tuple_bounding_school = adjusttotuples(new_bounding_school)

In [29]:
# new_bounding_college = applyandcopymergeways(try_tuple_bounding_college)
# new_bounding_university = applyandcopymergeways(try_tuple_bounding_university)
# new_bounding_local = applyandcopymergeways(try_tuple_bounding_local)

In [30]:
# tuple_bounding_college = adjusttotuples(new_bounding_college)
# tuple_bounding_university = adjusttotuples(new_bounding_university)
# tuple_bounding_local = adjusttotuples(new_bounding_local)

In [31]:
def splitbadlists(alist):
    import copy
    alist_copy = copy.deepcopy(alist)
    badrelcount = 0
    badreldups = 0
    badrelnames = set()
    badwaycount = 0
    badwaydups = 0
    badwaynames = set()
    finallist = []
    badlistindex = []
    for probindex, bound in enumerate(alist_copy):
        newnodelist = bound['nodelist']
        if bound['type'] == 'relation':
            firstmembers = []
            lastmembers = []
            #logging.debug(bound)
            mark = False
            for i, waylist in enumerate(newnodelist):
                front = tuple(waylist[0])
                back = tuple(waylist[-1])
                if front != back: 
                    badrelcount += 1
                    badrelnames.add(bound['name'])
                    flattened = list(arbitraryflatten(newnodelist))
                    latitudes = [element for i, element in enumerate(flattened) if i%2==0]
                    longitudes = [element for i, element in enumerate(flattened) if i%2!=0]
                    zippedlist = list(zip(latitudes, longitudes))
                    dups = [item for item, count in collections.Counter(zippedlist).items() if count > 1]
                    if dups:
                        badreldups += 1
                    mark = True
            if mark:
                badlistindex.append(probindex)
            else:
                finallist.append(copy.deepcopy(bound))
        if bound['type'] == 'way':
            front = tuple(newnodelist[0])
            back = tuple(newnodelist[-1])
            if front != back:
                badwaynames.add(bound['name'])
                badwaycount += 1
                dups = [item for item, count in collections.Counter(newnodelist).items() if count > 1]
                if dups:
                    badwaydups += 1
                    startindex = newnodelist.index(dups[0])
                    bound['nodelist'] = bound['nodelist'][startindex:] + bound['nodelist'][:startindex + 1]
                    finallist.append(copy.deepcopy(bound))
                else:
                    badlistindex.append(probindex)
            else:
                finallist.append(copy.deepcopy(bound))
            

    logging.debug(badrelnames)
    logging.debug(badwaynames)
    logging.debug("bad rel count: " + str(badrelcount))
    logging.debug("bad rel dups: " + str(badreldups))
    logging.debug("bad way count: " + str(badwaycount))
    logging.debug("bad way dups: " + str(badwaydups))
    return finallist, badlistindex

In [32]:
final_bounding_school, bad_bounding_school_indexes = splitbadlists(tuple_bounding_school)

{'Maltby Lilly Hall Junior School', 'Theresianische Akademie', 'Peter-Rosegger-Schule', 'Groupe Scolaire Janson de Sailly', 'Gimnazjum nr 1', 'Montgomery High School - A Language College and Full Service School', 'Istituto Comprensivo Cristoforo Madruzzo', 'Snouci Abdelkader primary school', 'Sophie-und-Hans-Scholl-Schule', 'Day Creek Intermediate School', 'Ünye Mustafa Rakım Anadolu Lisesi', 'Centre François Lepoire', 'Jardim de Infância & Escola Primária de Foterin Laconac', 'Institución Educativa Julio Restrepo Sede 2', 'Nelly-Sachs-IGS', 'Plummer Middle School', 'Dzivarasekwa 1 High School', 'Gayle Hamilton Gray Middle School', 'Szkoła Policji w Katowicach', 'École Cheikh Touré', 'Brisbane Grammar School', 'I.E.S. Julián Zarco', 'École Saint-Éloi', 'Karl-Gärtner-Schule', '海上自衛隊舞鶴教育隊', "Mount Saint Mary's Seminary", 'Maltby Comprehensive School', 'โรงเรียนบ้านโนนแดงโคกเล้า', 'Laerskool Swartland', '台北歐洲學校中學部', 'Ankeney Middle School', 'โรงเรียนเขาสวนกวางวิทยานุกูล', 'Lycée Charlie C

bad rel count: 228
bad rel dups: 223
bad way count: 330
bad way dups: 30


In [33]:
just_bad_bounding_school = copy.deepcopy([tuple_bounding_school[index] for index in bad_bounding_school_indexes])

In [34]:
# final_bounding_university, bad_bounding_university_indexes = splitbadlists(tuple_bounding_university)
# final_bounding_college, bad_bounding_college_indexes = splitbadlists(tuple_bounding_college)
# final_bounding_local, bad_bounding_local_indexes = splitbadlists(tuple_bounding_local)

In [35]:
# just_bad_bounding_university = copy.deepcopy([tuple_bounding_university[index] for index in bad_bounding_university_indexes])
# just_bad_bounding_college = copy.deepcopy([tuple_bounding_college[index] for index in bad_bounding_college_indexes])
# just_bad_bounding_local = copy.deepcopy([tuple_bounding_local[index] for index in bad_bounding_local_indexes])

In [36]:
import random
def forwardreversecomparisonrandomized1(thenodelist, randint):
    import copy
    bestscore = 0
    bestnodelist = []
    bestsafelist = []
    for i in range(randint):
        count = 0
        newnodelist = copy.deepcopy(thenodelist)
        safewaylists = []

        firstmembers = []
        lastmembers = []

        for index, waylist in enumerate(newnodelist):
            for wayindex, element in enumerate(waylist):
                waylist[wayindex] = tuple(element)

        for index, waylist in enumerate(newnodelist):
            if waylist[-1] == waylist[0]:
                safewaylists.append(copy.deepcopy(waylist))
                newnodelist.pop(index)
            else:
                firstmembers.append(copy.deepcopy(waylist[0]))
                lastmembers.append(copy.deepcopy(waylist[-1]))

        nextfirstmembers = []
        nextlastmembers = []

        while nextfirstmembers != firstmembers and nextlastmembers != lastmembers:
            nextfirstmembers = []
            nextlastmembers = []
            for index, waylist in enumerate(newnodelist):         
                if waylist[-1] == waylist[0]:
                    safewaylists.append(copy.deepcopy(waylist))
                    newnodelist.pop(index)
                else:
                    nextfirstmembers.append(copy.deepcopy(waylist[0]))
                    nextlastmembers.append(copy.deepcopy(waylist[-1]))

            firstmembers = copy.deepcopy(nextfirstmembers)
            lastmembers = copy.deepcopy(nextlastmembers)

            aset = set()
            for el in firstmembers:
                aset.add(el)
            shared = aset.intersection(lastmembers)

            while shared != set():
                count += 1
                share = shared.pop()
                index1 = firstmembers.index(share)
                index2 = lastmembers.index(share)

                newnodelist[index1].pop(0)

                newnodelist[index2].extend(copy.deepcopy(newnodelist[index1]))
                
                thechoice = random.choice([True, False])
                if thechoice:
                    newnodelist[index2] = newnodelist[index2][::-1]

                newnodelist.pop(index1)

                firstmembers = []
                lastmembers = []
                for index, waylist in enumerate(newnodelist):         
                    if waylist[-1] == waylist[0]:
                        safewaylists.append(copy.deepcopy(waylist))
                        newnodelist.pop(index)
                    else:
                        firstmembers.append(copy.deepcopy(waylist[0]))
                        lastmembers.append(copy.deepcopy(waylist[-1]))
                aset = set()
                for el in firstmembers:
                    aset.add(el)
                shared = aset.intersection(lastmembers)
        if count >= bestscore:
            bestscore = count
            bestnodelist = newnodelist
            bestsafelist = safewaylists
    return bestnodelist, bestsafelist, bestscore

In [37]:
def mergewaysrandomized1(alist, randint):
    import copy
    finalnodelist = []
    alist_copy = copy.deepcopy(alist)
    for progressindex, bound in enumerate(alist_copy):
        if progressindex % 100 == 0:
            logging.debug(progressindex)
        if bound['type'] == 'relation':
            newnodelist = bound['nodelist']
            safewaylists = []
            #######################
            firstmembers = []
            lastmembers = []
            for index, waylist in enumerate(newnodelist):
                if waylist[-1] == waylist[0]:
                    safewaylists.append(copy.deepcopy(waylist))
                    newnodelist.pop(index)
                else:
                    firstmembers.append(copy.deepcopy(waylist[0]))
                    lastmembers.append(copy.deepcopy(waylist[-1]))
                    
                    
            nextfirstmembers = []
            nextlastmembers = []
            #########################
            while nextfirstmembers != firstmembers and nextlastmembers != lastmembers:
               
                nextfirstmembers = []
                nextlastmembers = []
                for index, waylist in enumerate(newnodelist):         
                    if waylist[-1] == waylist[0]:
                        safewaylists.append(copy.deepcopy(waylist))
                        newnodelist.pop(index)
                    else:
                        nextfirstmembers.append(copy.deepcopy(waylist[0]))
                        nextlastmembers.append(copy.deepcopy(waylist[-1]))
                
                firstmembers = copy.deepcopy(nextfirstmembers)
                lastmembers = copy.deepcopy(nextlastmembers)
                
                ######################## firstmembers
                
                firstmemdups = [item for item, count in collections.Counter(firstmembers).items() if count > 1]
                while firstmemdups:
                    first = firstmemdups.pop()
                    
                    index1 = firstmembers.index(first)
                    index2 = [i for i, el in enumerate(firstmembers) if el == first][1]
                    
                    #$$$$$$$$$$$$ try both
                    templist1 = copy.deepcopy(newnodelist)
                    templist1[index1] = templist1[index1][::-1]
                    
                    templist1, templist1safe, templist1bestcount = forwardreversecomparisonrandomized1(templist1, randint)
                    
                    templist2 = copy.deepcopy(newnodelist)
                    templist2[index2] = templist2[index2][::-1]
                    
                    templist2, templist2safe, templist2bestcount = forwardreversecomparisonrandomized1(templist2, randint)
                    
                    if templist1bestcount >= templist2bestcount:
                        newnodelist = templist1
                        safewaylists.extend(templist1safe)
                    else:
                        newnodelist = templist2
                        safewaylists.extend(templist2safe)
                    
                    firstmembers = []
                    lastmembers = []
                    for index, waylist in enumerate(newnodelist):         
                        if waylist[-1] == waylist[0]:
                            safewaylists.append(copy.deepcopy(waylist))
                            newnodelist.pop(index)
                        else:
                            firstmembers.append(copy.deepcopy(waylist[0]))
                            lastmembers.append(copy.deepcopy(waylist[-1]))
                    firstmemdups = [item for item, count in collections.Counter(firstmembers).items() if count > 1]
                 
                
                #########
                firstmembers = []
                lastmembers = []
                for index, waylist in enumerate(newnodelist):         
                    if waylist[-1] == waylist[0]:
                        safewaylists.append(copy.deepcopy(waylist))
                        newnodelist.pop(index)
                    else:
                        firstmembers.append(copy.deepcopy(waylist[0]))
                        lastmembers.append(copy.deepcopy(waylist[-1]))
                ######################## lastmembers
                lastmemedups = [item for item, count in collections.Counter(lastmembers).items() if count > 1]
                while lastmemedups:
                    
                    last = lastmemedups.pop()
                    
                    index1 = lastmembers.index(last)
                    index2 = [i for i, el in enumerate(lastmembers) if el == last][1]


                    #$$$$$$$$$$$$ try both
                    templist1 = copy.deepcopy(newnodelist)
                    templist1[index1] = templist1[index1][::-1]
                    
                    templist1, templist1safe, templist1bestcount = forwardreversecomparisonrandomized1(templist1, randint)
                    
                    templist2 = copy.deepcopy(newnodelist)
                    templist2[index2] = templist2[index2][::-1]
                    
                    templist2, templist2safe, templist2bestcount = forwardreversecomparisonrandomized1(templist2, randint)
                    
                    if templist1bestcount >= templist2bestcount:
                        newnodelist = templist1
                        safewaylists.extend(templist1safe)
                    else:
                        newnodelist = templist2
                        safewaylists.extend(templist2safe)

                    firstmembers = []
                    lastmembers = []
                    for index, waylist in enumerate(newnodelist):         
                        if waylist[-1] == waylist[0]:
                            safewaylists.append(copy.deepcopy(waylist))
                            newnodelist.pop(index)
                        else:
                            firstmembers.append(copy.deepcopy(waylist[0]))
                            lastmembers.append(copy.deepcopy(waylist[-1]))
                    lastmemedups = [item for item, count in collections.Counter(lastmembers).items() if count > 1]
            newnodelist, safewaylists, _ = forwardreversecomparisonrandomized1(newnodelist, randint)
            newsafewaylists = []
            for safe in safewaylists:
              if safe not in newsafewaylists:
                newsafewaylists.append(safe)
            safewaylists = newsafewaylists
            newnodelist.extend(safewaylists)
            finalnodelist.append(newnodelist)
        if bound['type'] == 'way':
            nodelist = bound['nodelist']
            newnodelist = copy.deepcopy(nodelist)
            finalnodelist.append(newnodelist)
    return finalnodelist

In [38]:
def applyandcopymergewaysrandomized1(alist, randint):
    import copy
    alist_copy = copy.deepcopy(alist)
    allnodelists = mergewaysrandomized1(alist_copy, randint)
    for nodelistindex, nodelist in enumerate(allnodelists):
        logging.debug(nodelistindex)
        alist_copy[nodelistindex]['nodelist'] = nodelist
        if len(nodelist) > 1:
            alist_copy[nodelistindex]['type'] = 'relation'
            try:
                alist_copy[nodelistindex]['nodelist'][0][0][0]
            except TypeError as notfullynested:
                alist_copy[nodelistindex]['type'] = 'way'
        elif len(nodelist) == 1 and alist_copy[nodelistindex]['type'] == 'relation':
            alist_copy[nodelistindex]['nodelist'] = nodelist[0]
            alist_copy[nodelistindex]['type'] = 'way'
    return alist_copy

In [39]:
attemptfix1_just_bad_bounding_school = applyandcopymergewaysrandomized1(just_bad_bounding_school, 1000)


0
100
200
300
400
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


In [40]:
bads1_final_bounding_school, bad1_2x_bounding_school_indexes = splitbadlists(attemptfix1_just_bad_bounding_school)


{'Summit High School', 'Ecole Primaire 1er November', 'Theresianische Akademie', 'Школа № 1', 'Groupe Scolaire Janson de Sailly', 'Banyan Elementary School', '临沂市第四中学', 'San Pascual Elementary School', 'École maternelle et élémentaire Gustave Roch', 'Gimnazjum nr 1', 'Oberschule Herzberg am Harz', 'Montgomery High School - A Language College and Full Service School', 'Vincenti Grundschule', 'Menntaskólinn í Reykjavík', 'Maltby Manor Primary School', 'Technische Scholen Mechelen', 'Sophie-und-Hans-Scholl-Schule', 'Day Creek Intermediate School', 'IPARM (434)', '朴园小学', 'Landkreis Schulzentrum', 'Maltby Comprehensive School', 'St.-Christophorus-Gymnasium', 'Cesar Vallejo', 'Albrecht-Dürer-Schule', 'CEM SADOKI SADEK', 'Sheth Chimanlal Nagindas Vidhyavihar', 'Nelly-Sachs-IGS', '石门实验学校', '2458114', 'École primaire catholique Jean XXIII - Établissement privé', 'Groenendaal Campus Sint-Jan', 'Школа №95', 'Gymnasium Grafing', 'Theodor Fliedner Gesamtschule', 'École Cheikh Touré', 'Brisbane Gram

bad rel count: 152
bad rel dups: 119
bad way count: 354
bad way dups: 13


In [41]:
# attemptfix1_just_bad_bounding_university = applyandcopymergewaysrandomized1(just_bad_bounding_university, 1000)
# attemptfix1_just_bad_bounding_college = applyandcopymergewaysrandomized1(just_bad_bounding_college, 1000)
# attemptfix1_just_bad_bounding_local = applyandcopymergewaysrandomized1(just_bad_bounding_local, 1000)

In [42]:
# bads1_final_bounding_university, bad1_2x_bounding_university_indexes = splitbadlists(attemptfix1_just_bad_bounding_university)
# bads1_final_bounding_college, bad1_2x_bounding_college_indexes = splitbadlists(attemptfix1_just_bad_bounding_college)
# bads1_final_bounding_local, bad1_2x_bounding_local_indexes = splitbadlists(attemptfix1_just_bad_bounding_local)

In [43]:
final_bounding_school.extend(bads1_final_bounding_school)


In [44]:
just_bad2x_bounding_school = copy.deepcopy([attemptfix1_just_bad_bounding_school[index] for index in bad1_2x_bounding_school_indexes])


In [45]:
# final_bounding_university.extend(bads1_final_bounding_university)
# final_bounding_college.extend(bads1_final_bounding_college)
# final_bounding_local.extend(bads1_final_bounding_local)

In [46]:
# just_bad2x_bounding_university = copy.deepcopy([attemptfix1_just_bad_bounding_university[index] for index in bad1_2x_bounding_university_indexes])
# just_bad2x_bounding_college = copy.deepcopy([attemptfix1_just_bad_bounding_college[index] for index in bad1_2x_bounding_college_indexes])
# just_bad2x_bounding_local = copy.deepcopy([attemptfix1_just_bad_bounding_local[index] for index in bad1_2x_bounding_local_indexes])

In [47]:
def removedups(alist):
    alist_copy = copy.deepcopy(alist)
    seenbound = []
    dupbound = 0
    dupway = 0
    new_alist_copy = [bound for bindex,bound in enumerate(alist_copy) if bound not in alist_copy[:bindex]]
    for bindex, bound in enumerate(new_alist_copy):
        if bound['type'] == 'relation':
            copy_nodelist = copy.deepcopy(bound['nodelist'])
            new_nodelist = [waylist for windex,waylist in enumerate(copy_nodelist) if waylist not in copy_nodelist[:windex]]
            bound['nodelist'] = new_nodelist
    for bindex, bound in enumerate(new_alist_copy):
        if bound not in seenbound:
            seenbound.append(bound)
        else:
            dupbound += 1
        if bound['type'] == 'relation':
            seenwaylist = []
            for waylist in bound['nodelist']:
                if waylist not in seenwaylist:
                    seenwaylist.append(waylist)
                else:
                    dupway+=1
    logging.debug(dupbound)
    logging.debug(dupway)
    return new_alist_copy

In [48]:
final_nodups_bounding_school = removedups(final_bounding_school)


0
0


In [49]:
just_bad2x_bounding_school = removedups(just_bad2x_bounding_school)


0
0


In [50]:
# final_nodups_bounding_university = removedups(final_bounding_university)
# final_nodups_bounding_college = removedups(final_bounding_college)
# final_nodups_bounding_local = removedups(final_bounding_local)

In [51]:
# just_bad2x_bounding_university = removedups(just_bad2x_bounding_university)
# just_bad2x_bounding_college = removedups(just_bad2x_bounding_college)
# just_bad2x_bounding_local = removedups(just_bad2x_bounding_local)

In [52]:
with open(bounding_school_parsed_path[:-5] + '_final.json', 'w') as write_file:
    json.dump(final_nodups_bounding_school, write_file, indent = 4)
with open(bounding_school_parsed_path[:-5] + '_bad.json', 'w') as write_file:
    json.dump(just_bad2x_bounding_school, write_file, indent = 4)

In [30]:
# with open(bounding_local_parsed_path[:-5] + '_final.json', 'w') as write_file:
#     json.dump(final_nodups_bounding_local, write_file, indent = 4)
# with open(bounding_university_parsed_path[:-5] + '_final.json', 'w') as write_file:
#     json.dump(final_nodups_bounding_university, write_file, indent = 4)
# with open(bounding_college_parsed_path[:-5] + '_final.json', 'w') as write_file:
#     json.dump(final_nodups_bounding_college, write_file, indent = 4)
# with open(bounding_university_parsed_path[:-5] + '_bad.json', 'w') as write_file:
#     json.dump(just_bad2x_bounding_university, write_file, indent = 4)
# with open(bounding_college_parsed_path[:-5] + '_bad.json', 'w') as write_file:
#     json.dump(just_bad2x_bounding_college, write_file, indent = 4)
